<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Push Swap método "Modern Times"
En honor a la película de 1936 de Chaplin.

In [ ]:
# FUNCIONES

def lista_ordenada(lis, orden_inverso=False): # la función da True si ponemos lista_ordenada([4,3,2,1], orden_inverso=True)
    ordenada = False
    aux = lis[:]
    aux.sort(reverse=orden_inverso)
    if aux == lis:
        ordenada = True
    return ordenada

def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

In [ ]:
# Generación de la pila A
from random import sample, seed
seed()
n = 100   # número de elementos de la pila
#a = sample(range(1, n+1), n)
a = [35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]
print('a:', a)
b = []
a_original = a[:]
n = len(a)

a: [35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]


In [ ]:
def algoritmo_LIS(arr):
    n = len(arr)
    lis = [1]*n
    prev = list(range(n))  # una lista con los index
    
    # Compute optimized LIS values in bottom up manner
    for i in range (1, n):
        for j in range(i):
            if arr[i] > arr[j] and lis[i] < lis[j] + 1:
                lis[i] = lis[j]+1
                prev[i] = j

    # Initialize maximum to 0 to get the maximum of all
    # LIS
    maximum = 0
    idx = 0

    # Pick maximum of all LIS values
    for i in range(n):
        if maximum < lis[i]:
            maximum = lis[i]
            idx = i

    seq = [arr[idx]]
    while idx != prev[idx]:
        idx = prev[idx]
        seq.append(arr[idx])

    return (maximum, list(reversed(seq)))

(nlis, lis) = algoritmo_LIS(a)   #nlist es el número de elementos de la lista lis
print(f"a_original:\n{a_original}")
print("nlis:", nlis)
print(f"LIS:\n{lis}")

a_original:
[35, 90, 6, 15, 7, 40, 85, 63, 50, 13, 45, 8, 76, 33, 25, 27, 42, 20, 94, 48, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 55, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 56, 70, 22, 11, 59, 68, 100, 96, 71, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 74, 23, 5, 9, 43, 49, 37, 80, 18, 17, 67, 83, 78, 61, 47, 57, 64, 81, 97, 53, 65, 30, 2, 99, 14, 87, 39, 19, 72, 4, 62, 41]
nlis: 17
LIS:
[6, 7, 13, 25, 27, 42, 48, 55, 56, 59, 68, 71, 74, 80, 83, 97, 99]


In [ ]:
# Llevamos de la pila A a la B los elementos de la LIS
contador = 0
# pasos necesarios para situarse como el primer elemento de la lista a
for vlis in lis:   # vlis es una variable que recoge cada uno de los elemento de la lista lis
    optimo = 2**31   # virtualmente infinito, se usa para luego calcular el min
    elemento_optimo = None
    for va in a:   # va es cada valor de la lista a
        if va == vlis:
            izquierda = a.index(va)
            derecha = len(a) - a.index(va)
            mejor_lado = min(izquierda, derecha)
            if mejor_lado < optimo:
                optimo = mejor_lado
                elemento_optimo = va

    if a.index(elemento_optimo) <= len(a) - a.index(elemento_optimo):   # si la izquierda es <= que la derecha
        for i in range(optimo):
            ra(a,b); contador += 1
        pb(a,b)
    else:
        for i in range(optimo):
            rra(a,b); contador += 1
        pb(a,b)
        

print(a)
print(b)
print(contador)

[14, 87, 39, 19, 72, 4, 62, 41, 35, 90, 15, 40, 85, 63, 50, 45, 8, 76, 33, 20, 94, 91, 29, 95, 77, 82, 98, 86, 69, 60, 58, 79, 24, 12, 28, 93, 73, 54, 84, 44, 10, 75, 3, 32, 89, 52, 51, 16, 70, 22, 11, 100, 96, 21, 26, 38, 1, 46, 92, 36, 88, 31, 66, 34, 23, 5, 9, 43, 49, 37, 18, 17, 67, 78, 61, 47, 57, 64, 81, 53, 65, 30, 2]
[99, 97, 83, 80, 74, 71, 68, 59, 56, 55, 48, 42, 27, 25, 13, 7, 6]
75


In [ ]:
# PASOS NECESARIOS PARA COLOCAR CADA ELEMENTO DE A EN SU SITIO DE B
# PASOS NECESARIOS = Pasos necesarios A + Pasos necesarios B

# Pasos necesarios para situarse como el primer elemento de la pila A
pasosA = []   # [0,1,2, .... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B

